In [3]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import random
from dataclasses import dataclass
from typing import List, Dict, Tuple

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline

from sklearn.ensemble import (
    RandomForestClassifier,
    IsolationForest,
    GradientBoostingRegressor,
    GradientBoostingClassifier,
)
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans

from sklearn.metrics import (
    classification_report, confusion_matrix,
    precision_score, recall_score, f1_score,
    mean_absolute_error
)

In [4]:
import pandas as pd

df = pd.read_csv('/content/invoices_enriched_final.csv')
display(df.head())

,first_name,last_name,email,product_id,qty,amount,invoice_date,address,city,stock_code,job,unit_price,avg_price,cost_price,list_price,discount_pct,margin_pct,salesperson,invoice_delay
0,Carmen Nixon,Todd Anderson,marvinjackson@example.com,133,9,14.57,10/09/1982,283 Wendy Common,West Alexander,36239634,Logistics and distribution manager,1.62,17.16,12.87,21.45,92.45,-87.42,Salesperson_07,0
1,Mrs. Heather Miller,Julia Moore,jeffrey84@example.net,155,5,65.48,03/10/2012,13567 Patricia Circles Apt. 751,Andreamouth,2820163,Osteopath,13.10,15.41,11.55,19.26,31.99,13.34,Salesperson_20,7
2,Crystal May,Philip Moody,ugoodman@example.com,151,9,24.66,23/03/1976,6389 Debbie Island Suite 470,Coxbury,27006726,Economist,2.74,13.51,10.13,16.89,83.77,-72.96,Salesperson_15,4
3,Bobby Weber,Mark Scott,ssanchez@example.com,143,4,21.34,17/08/1986,6362 Ashley Plaza Apt. 994,Ninaland,83036521,Sports administrator,5.34,15.28,11.46,19.10,72.07,-53.44,Salesperson_11,2
4,Kristen Welch,David David,cynthia66@example.net,168,2,83.90,11/06/1996,463 Steven Cliffs Suite 757,Isaiahview,80142652,Chief Marketing Officer,41.95,15.87,11.90,19.84,111.49,252.48,Salesperson_08,0


In [5]:
@dataclass
class ModelConfig:
    random_state: int = 42
    test_size: float = 0.2
    cv_folds: int = 5

    iso_estimators: int = 200
    iso_contamination: float = 0.05

    rf_estimators: int = 200
    rf_max_depth: int = 10

    gbr_estimators: int = 200
    gbr_learning_rate: float = 0.05
    gbr_max_depth: int = 5

In [6]:
@dataclass
class RiskConfig:
    weight_supervised: float = 0.4
    weight_iso: float = 0.3
    weight_price: float = 0.3
    base_threshold: float = 0.35
    precision_k_list: List[int] = None

    def __post_init__(self):
        if self.precision_k_list is None:
            self.precision_k_list = [10, 20, 50, 100]

In [8]:
def clean_and_engineer(df: pd.DataFrame) -> pd.DataFrame:
  df = df.copy()

  # Drop clearly irrelevant columns
  drop_cols = ["first_name", "last_name", "email", "address", "job"]
  drop_cols = [c for c in drop_cols if c in df.columns]
  df = df.drop(columns=drop_cols, errors="ignore")

  # Parse date
  if "invoice_date" in df.columns:
      df["invoice_date"] = pd.to_datetime(df["invoice_date"], errors="coerce")

  # Ensure core numeric columns are numeric
  numeric_cols = [
      "product_id", "qty", "amount",
      "stock_code",
      "unit_price", "avg_price", "cost_price", "list_price",
      "discount_pct", "margin_pct",
      "invoice_delay"
  ]
  for col in numeric_cols:
      if col in df.columns:
          df[col] = pd.to_numeric(df[col], errors="coerce")

  # Handle list_price zeros
  if "list_price" in df.columns:
      df["list_price"] = df["list_price"].replace(0, np.nan)

  # Fill basic numeric nulls with median
  for col in numeric_cols:
      if col in df.columns:
          df[col] = df[col].fillna(df[col].median())

  # ---------- Feature Engineering ----------
  # price_gap: difference between actual unit price and avg_price
  if "unit_price" in df.columns and "avg_price" in df.columns:
      df["price_gap"] = df["unit_price"] - df["avg_price"]
      df["price_gap_pct"] = np.where(
          df["avg_price"] != 0,
          (df["unit_price"] - df["avg_price"]) / df["avg_price"],
          0.0
      )
  else:
      df["price_gap"] = 0.0
      df["price_gap_pct"] = 0.0

  # expected_margin based on avg_price and cost_price
  if "avg_price" in df.columns and "cost_price" in df.columns:
      df["expected_margin"] = np.where(
          df["avg_price"] != 0,
          (df["avg_price"] - df["cost_price"]) / df["avg_price"],
          0.0
      )
  else:
      df["expected_margin"] = 0.0

  # margin_gap = expected_margin - actual margin_pct
  if "margin_pct" in df.columns:
      df["margin_gap"] = df["expected_margin"] - df["margin_pct"]
  else:
      df["margin_gap"] = 0.0

  # Fill engineered features for safety
  for col in ["price_gap", "price_gap_pct", "expected_margin", "margin_gap"]:
      df[col] = df[col].replace([np.inf, -np.inf], np.nan).fillna(0.0)

  return df


In [9]:
def apply_business_rules(df: pd.DataFrame) -> pd.DataFrame:
    """
    Rule logic:
        - High discount: discount_pct > 0.25
        - Underpricing: price_gap_pct < -0.20  (selling 20% below avg)
        - Margin collapse: margin_gap > 0.20  (expected margin 20% higher than realised)
        - Delay: invoice_delay > 30 days
    """
    df = df.copy()

    # Safe defaults
    if "discount_pct" not in df.columns:
        df["discount_pct"] = 0.0
    if "price_gap_pct" not in df.columns:
        df["price_gap_pct"] = 0.0
    if "margin_gap" not in df.columns:
        df["margin_gap"] = 0.0
    if "invoice_delay" not in df.columns:
        df["invoice_delay"] = 0

    df["rule_high_discount"] = (df["discount_pct"] > 0.25).astype(int)
    df["rule_underpricing"] = (df["price_gap_pct"] < -0.20).astype(int)
    df["rule_margin_collapse"] = (df["margin_gap"] > 0.20).astype(int)
    df["rule_high_delay"] = (df["invoice_delay"] > 30).astype(int)

    df["rule_based_leak"] = (
        df["rule_high_discount"] |
        df["rule_underpricing"] |
        df["rule_margin_collapse"] |
        df["rule_high_delay"]
    ).astype(int)

    # If explicit revenue_leak not present, create it from rules
    if "revenue_leak" not in df.columns:
        df["revenue_leak"] = df["rule_based_leak"]

    return df


In [10]:
def get_feature_columns(df: pd.DataFrame) -> Tuple[List[str], List[str]]:
    """
    Define numeric and categorical features based on the known schema.
    """
    numeric_features = [
        "qty", "amount",
        "unit_price", "avg_price", "cost_price", "list_price",
        "discount_pct", "margin_pct",
        "invoice_delay",
        "price_gap", "price_gap_pct",
        "expected_margin", "margin_gap"
    ]
    numeric_features = [c for c in numeric_features if c in df.columns]

    categorical_features = [
        "product_id", "city",
        "stock_code",
        "salesperson"
    ]
    categorical_features = [c for c in categorical_features if c in df.columns]

    return numeric_features, categorical_features


In [12]:
def build_feature_matrix(df: pd.DataFrame):
    """
    Build X, y and a ColumnTransformer for scaling + encoding.
    """
    numeric_features, categorical_features = get_feature_columns(df)

    # Label
    if "revenue_leak" not in df.columns:
        raise ValueError("revenue_leak column is required at this stage.")

    y = df["revenue_leak"].astype(int)
    X = df[numeric_features + categorical_features].copy()

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), numeric_features),
            ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ]
    )

    return X, y, preprocessor, numeric_features, categorical_features

SUPERVISED CLASSIFIER – WITH MODEL COMPARISON

In [18]:
def train_and_compare_classifiers(
    X,
    y,
    preprocessor,
    cfg: ModelConfig
) -> Tuple[Pipeline, Dict]:
    """
    Compare multiple supervised models on CV + test,
    pick best by F1 on test set.
    Models:
        - RandomForestClassifier
        - LogisticRegression
        - GradientBoostingClassifier
    """
    models = {
        "RandomForest": RandomForestClassifier(
            n_estimators=cfg.rf_estimators,
            max_depth=cfg.rf_max_depth,
            class_weight="balanced",
            random_state=cfg.random_state
        ),
        "LogisticRegression": LogisticRegression(
            max_iter=1000,
            class_weight="balanced",
            random_state=cfg.random_state
        ),
        "GradientBoostingClassifier": GradientBoostingClassifier(
            n_estimators=150,
            learning_rate=0.05,
            max_depth=3,
            random_state=cfg.random_state
        ),
    }

    results = {}
    best_model_name = None
    best_f1 = -1.0
    best_pipe = None

    # Single train-test split shared across models for fair comparison
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=cfg.test_size,
        stratify=y,
        random_state=cfg.random_state
    )

    for name, model in models.items():
        pipe = Pipeline(steps=[
            ("preprocess", preprocessor),
            ("clf", model)
        ])

        # Cross-validation F1
        cv_scores = cross_val_score(pipe, X, y, cv=cfg.cv_folds, scoring="f1")
        cv_mean = float(cv_scores.mean())
        cv_std = float(cv_scores.std())

        # Fit on train and evaluate on test
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)

        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        report = classification_report(y_test, y_pred, output_dict=True)
        cm = confusion_matrix(y_test, y_pred).tolist()

        results[name] = {
            "cv_f1_mean": cv_mean,
            "cv_f1_std": cv_std,
            "precision_test": float(prec),
            "recall_test": float(rec),
            "f1_test": float(f1),
            "classification_report": report,
            "confusion_matrix": cm
        }

        if f1 > best_f1:
            best_f1 = f1
            best_model_name = name
            best_pipe = pipe

    print("\n=== Supervised Model Comparison (Test F1) ===")
    for name, res in results.items():
        print(
            f"{name}: F1_test={res['f1_test']:.3f}, "
            f"Precision={res['precision_test']:.3f}, "
            f"Recall={res['recall_test']:.3f}, "
            f"CV_F1={res['cv_f1_mean']:.3f}±{res['cv_f1_std']:.3f}"
        )

    print(f"\nSelected best model for deployment: {best_model_name} (F1={best_f1:.3f})")

    return best_pipe, {"all_models": results, "best_model": best_model_name}


ANOMALY MODELS

In [19]:
def fit_isolation_forest(df: pd.DataFrame, cfg: ModelConfig) -> pd.DataFrame:
    """
    Use core numeric and engineered leak-related features for Isolation Forest.
    """
    df = df.copy()
    anomaly_features = [
        "qty", "amount",
        "unit_price", "avg_price", "cost_price", "list_price",
        "discount_pct", "margin_pct",
        "invoice_delay",
        "price_gap", "price_gap_pct",
        "expected_margin", "margin_gap"
    ]
    anomaly_features = [c for c in anomaly_features if c in df.columns]

    if not anomaly_features:
        df["iso_anomaly"] = 0
        return df

    scaler = StandardScaler()
    X_iso = scaler.fit_transform(df[anomaly_features].fillna(0))

    iso = IsolationForest(
        n_estimators=cfg.iso_estimators,
        contamination=cfg.iso_contamination,
        random_state=cfg.random_state
    )
    preds = iso.fit_predict(X_iso)
    df["iso_anomaly"] = pd.Series(preds).map({1: 0, -1: 1}).astype(int)

    return df


def train_price_regressor(df: pd.DataFrame, X, preprocessor, cfg: ModelConfig):
    """
    Predict unit_price, compute deviation and price_anomaly.
    """
    df = df.copy()
    if "unit_price" not in df.columns:
        df["predicted_price"] = np.nan
        df["price_deviation_model"] = 0.0
        df["price_anomaly"] = 0
        return None, 0.0, df

    y_reg = df["unit_price"]

    reg = GradientBoostingRegressor(
        n_estimators=cfg.gbr_estimators,
        learning_rate=cfg.gbr_learning_rate,
        max_depth=cfg.gbr_max_depth,
        random_state=cfg.random_state
    )

    pipe = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("reg", reg)
    ])

    X_train, X_test, y_train, y_test = train_test_split(
        X, y_reg,
        test_size=cfg.test_size,
        random_state=cfg.random_state
    )

    pipe.fit(X_train, y_train)
    y_pred_test = pipe.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred_test)

    # Predictions on full data
    y_pred_full = pipe.predict(X)
    df["predicted_price"] = y_pred_full
    df["price_deviation_model"] = (df["unit_price"] - df["predicted_price"]).abs()
    df["price_anomaly"] = (df["price_deviation_model"] > 2 * mae).astype(int)

    return pipe, float(mae), df

CLUSTERING – SALESPERSON

In [20]:
def cluster_salespersons(df: pd.DataFrame, n_clusters: int = 4) -> pd.DataFrame:
    df = df.copy()
    if "salesperson" not in df.columns:
        df["sales_cluster"] = -1
        return df

    agg = df.groupby("salesperson").agg({
        "discount_pct": "mean",
        "margin_pct": "mean",
        "price_gap_pct": "mean",
        "invoice_delay": "mean",
        "qty": "mean",
        "unit_price": "mean",
        "amount": "mean"
    }).fillna(0)

    features_for_cluster = [
        c for c in ["discount_pct", "margin_pct", "price_gap_pct",
                    "invoice_delay", "qty", "unit_price", "amount"]
        if c in agg.columns
    ]

    if len(features_for_cluster) == 0:
        df["sales_cluster"] = -1
        return df

    scaler = StandardScaler()
    X_cluster = scaler.fit_transform(agg[features_for_cluster])

    if agg.shape[0] < n_clusters:
        n_clusters = max(2, agg.shape[0])  # fallback

    km = KMeans(n_clusters=n_clusters, random_state=42, n_init="auto")
    agg["sales_cluster"] = km.fit_predict(X_cluster)

    cluster_map = agg["sales_cluster"].to_dict()
    df["sales_cluster"] = df["salesperson"].map(cluster_map)

    return df


RISK SCORING & EVALUATION

In [23]:
def compute_risk_score(df: pd.DataFrame, risk_cfg: RiskConfig) -> pd.DataFrame:
    df = df.copy()
    for col in ["ml_pred", "iso_anomaly", "price_anomaly"]:
        if col not in df.columns:
            df[col] = 0
        df[col] = df[col].fillna(0).astype(float)

    df["risk_score"] = (
        risk_cfg.weight_supervised * df["ml_pred"] +
        risk_cfg.weight_iso * df["iso_anomaly"] +
        risk_cfg.weight_price * df["price_anomaly"]
    )

    df["risk_flag"] = (df["risk_score"] >= risk_cfg.base_threshold).astype(int)
    return df

In [24]:
def precision_at_k(df: pd.DataFrame,
                   y_col: str,
                   score_col: str,
                   k_list: List[int]) -> Dict[int, float]:
    df_sorted = df.sort_values(score_col, ascending=False)
    results = {}
    for k in k_list:
        top_k = df_sorted.head(k)
        if len(top_k) == 0:
            results[k] = 0.0
        else:
            results[k] = float(top_k[y_col].mean())
    return results

In [25]:
def threshold_sweep(df: pd.DataFrame,
                    y_col: str,
                    score_col: str,
                    thresholds: List[float]) -> pd.DataFrame:
    rows = []
    for t in thresholds:
        preds = (df[score_col] >= t).astype(int)
        prec = precision_score(df[y_col], preds, zero_division=0)
        rec = recall_score(df[y_col], preds, zero_division=0)
        f1 = f1_score(df[y_col], preds, zero_division=0)
        flagged_pct = float(preds.mean())
        rows.append({
            "threshold": t,
            "precision": prec,
            "recall": rec,
            "f1": f1,
            "flagged_pct": flagged_pct
        })
    return pd.DataFrame(rows)


def cluster_leak_density(df: pd.DataFrame) -> pd.DataFrame:
    if "sales_cluster" not in df.columns or "revenue_leak" not in df.columns:
        return pd.DataFrame()
    return (df
            .groupby("sales_cluster")["revenue_leak"]
            .mean()
            .reset_index(name="leak_rate"))

MAIN ORCHESTRATION

In [27]:
OUTPUT_PATH = '/content/invoices_enriched_output.csv'

def main():
    # Reproducibility
    np.random.seed(42)
    random.seed(42)

    model_cfg = ModelConfig()
    risk_cfg = RiskConfig()

    # 1. Load
    df = df = pd.read_csv('/content/invoices_enriched_final.csv')

    # 2. Clean & engineer features
    df = clean_and_engineer(df)

    # 3. Business rules & labels
    df = apply_business_rules(df)

    # 4. Build feature matrix
    X, y, preprocessor, num_feats, cat_feats = build_feature_matrix(df)

    # 5. Supervised classifier WITH model comparison
    best_clf_pipe, clf_eval = train_and_compare_classifiers(X, y, preprocessor, model_cfg)
    best_model_name = clf_eval["best_model"]

    print("\n=== Deployed Supervised Model Details ===")
    print("Best model selected:", best_model_name)
    print("Full metrics per model are available in clf_eval['all_models'] (in code).")

    # predictions for all rows using best model
    df["ml_pred"] = best_clf_pipe.predict(X)

    # 6. Isolation Forest
    df = fit_isolation_forest(df, model_cfg)

    # 7. Price regressor anomaly
    reg_pipe, mae, df = train_price_regressor(df, X, preprocessor, model_cfg)
    print(f"\nPrice regressor MAE: {mae:.4f}")

    # 8. Salesperson clustering
    df = cluster_salespersons(df, n_clusters=4)

    # 9. Risk scoring
    df = compute_risk_score(df, risk_cfg)

    # 10. Evaluation on risk_score
    print("\n=== Precision@K (risk_score) ===")
    p_at_k = precision_at_k(df, "revenue_leak", "risk_score", risk_cfg.precision_k_list)
    for k, v in p_at_k.items():
        print(f"Precision@{k}: {v:.3f}")

    thresholds = [round(x, 2) for x in np.linspace(0.1, 0.9, 9)]
    sweep_df = threshold_sweep(df, "revenue_leak", "risk_score", thresholds)
    print("\n=== Threshold Sweep (risk_score) ===")
    print(sweep_df)

    clust_df = cluster_leak_density(df)
    if not clust_df.empty:
        print("\n=== Leak Density by Salesperson Cluster ===")
        print(clust_df)

    # 11. Executive summary
    total_invoices = len(df)
    flagged = int(df["risk_flag"].sum())
    flagged_amount = df.loc[df["risk_flag"] == 1, "amount"].sum() if "amount" in df.columns else np.nan

    print("\n=== Executive Summary ===")
    print(f"Total invoices: {total_invoices}")
    print(f"Invoices flagged as high-risk: {flagged} ({flagged / total_invoices:.2%})")
    if not np.isnan(flagged_amount):
        print(f"Total billed amount on high-risk invoices: {flagged_amount:,.2f}")

    # 12. Save enriched dataset
    df.to_csv(OUTPUT_PATH, index=False)
    print(f"\nEnriched file saved to {OUTPUT_PATH}")


if __name__ == "__main__":
    main()


=== Supervised Model Comparison (Test F1) ===
RandomForest: F1_test=0.999, Precision=0.998, Recall=1.000, CV_F1=0.999±0.002
LogisticRegression: F1_test=0.998, Precision=0.998, Recall=0.997, CV_F1=0.999±0.000
GradientBoostingClassifier: F1_test=1.000, Precision=1.000, Recall=1.000, CV_F1=1.000±0.000

Selected best model for deployment: GradientBoostingClassifier (F1=1.000)

=== Deployed Supervised Model Details ===
Best model selected: GradientBoostingClassifier
Full metrics per model are available in clf_eval['all_models'] (in code).

Price regressor MAE: 0.0159

=== Precision@K (risk_score) ===
Precision@10: 1.000
Precision@20: 1.000
Precision@50: 1.000
Precision@100: 1.000

=== Threshold Sweep (risk_score) ===
   threshold  precision    recall        f1  flagged_pct
0        0.1     0.9998  1.000000  0.999900       0.9984
1        0.2     0.9998  1.000000  0.999900       0.9984
2        0.3     0.9998  1.000000  0.999900       0.9984
3        0.4     1.0000  1.000000  1.000000      